<a href="https://colab.research.google.com/github/yashraj01-se/Machine_learning_important_concepts/blob/main/Naive_Bayes_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from collections import Counter

#step 1: Dataset :

In [11]:
docs  = ["free money offer",
         "win money win",
         "meeting schedule",
         "project meeting",
         "meeting on time",
         "free money to you",
         "free free free",
         "meeting is reschedule",
         "meeting is must",
         "free of cost"]
labels=torch.tensor([1,1,0,0,0,1,1,0,0,1],dtype=torch.int) #1 spam 0 not spam
vocab=["free","money","meeting","schedule","offer"]

word2idx={w:i for i,w in enumerate(vocab)}

#step 2: Bag of vector :
#vec → [0, 1, 2, 0, 1]  
#        ↑ ↑ ↑    ↑
#      free win money meeting offer

In [16]:
def bow_vector(text):
  vec=torch.zeros(len(vocab))
  for w in text.split():
    if w in word2idx:
      vec[word2idx[w]]+=1
  return vec

#Emails converted to tensor:

In [17]:
X = torch.stack([bow_vector(d) for d in docs])   # shape (4 emails, 5 words)

In [18]:
print(X)

tensor([[1., 1., 0., 0., 1.],
        [0., 1., 0., 0., 0.],
        [0., 0., 1., 1., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 1., 0., 0., 0.],
        [3., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 1., 0., 0.],
        [1., 0., 0., 0., 0.]])


#Laplace smoothing to avid unseen word prob:

In [23]:
alpha=1.0
V=X.size(1)

#calculated the priors:

In [37]:
class_count=torch.bincount(labels)
proirs=class_count/class_count.sum()
print(proirs)

tensor([0.5000, 0.5000])


In [25]:
spam_mask=labels==1
non_spam_mask=labels==0

#calculating the likelihood:

In [39]:
word_cound_spam=X[spam_mask].sum(0)+ alpha #this is a vector
word_cound_non_spam=X[non_spam_mask].sum(0)+alpha #this is a vector

likelihood_spam=word_cound_spam/word_cound_spam.sum()
likelihood_not_spam=word_cound_non_spam/word_cound_non_spam.sum()

print(word_cound_spam)
print(word_cound_non_spam)
print(word_cound_spam.sum())
print(word_cound_non_spam.sum())
print(likelihood_spam)
print(likelihood_not_spam)

tensor([7., 4., 1., 1., 2.])
tensor([1., 1., 6., 2., 1.])
tensor(15.)
tensor(11.)
tensor([0.4667, 0.2667, 0.0667, 0.0667, 0.1333])
tensor([0.0909, 0.0909, 0.5455, 0.1818, 0.0909])


#classifying the email:

In [45]:
def predict(text):
  x=bow_vector(text)
  #comput the log posterior of each class:
  log_pos_spam=(x* likelihood_spam.log()).sum()+proirs[1].log()
  log_pos_not_spam=(x*likelihood_not_spam.log()).sum()+proirs[0].log()

  return "This is a Spam Email. Be Aware" if log_pos_spam>log_pos_not_spam else "Important Email"

In [44]:
predict("Free invitation for money")

'This is a Spam Email. Be Aware'